# Finetuning llama2 models - from turorial

[Tutorial](https://blog.ovhcloud.com/fine-tuning-llama-2-models-using-a-single-gpu-qlora-and-ai-notebooks/)

# Fine-Tuning LLaMA 2 Models using a single GPU, QLoRA and AI Notebooks

*This tutorial walks through the process of fine-tuning [LLaMA 2](https://ai.meta.com/llama/) models, providing step-by-step instructions.*

*All the code related to this article is available in our dedicated [GitHub repository](https://github.com/ovh/ai-training-examples/blob/main/notebooks/natural-language-processing/llm/miniconda/llama2-fine-tuning/llama_2_finetuning.ipynb).*

## Introduction
On July 18, 2023, [Meta](https://about.meta.com/) released LLaMA 2, the latest version of their **Large Language Model** (LLM).

Trained between January 2023 and July 2023 on 2 trillion tokens, these new models outperforms other LLMs on many benchmarks, including reasoning, coding, proficiency, and knowledge tests. This release comes in different flavors, with parameter sizes of **[7B](https://huggingface.co/meta-llama/Llama-2-7b-hf)**, **[13B](https://huggingface.co/meta-llama/Llama-2-13b-hf)** and a mind-blowing **[70B](https://huggingface.co/meta-llama/Llama-2-70b-hf)**. Models are intended for free for both commercial and research use in English.

To suit every text generation needed and fine-tune these models, we will use [QLoRA](https://arxiv.org/abs/2305.14314) (Efficient Finetuning of Quantized LLMs), a highly efficient fine-tuning technique that involves quantizing a pretrained LLM to just 4 bits and adding small “Low-Rank Adapters”. This unique approach allows for fine-tuning LLMs **using just a single GPU**! This technique is supported by the [PEFT](https://huggingface.co/docs/peft/) library.

## Set up Python environment
The following libraries are used for this method (`requirements.txt` file):

```
torch
accelerate @ git+https://github.com/huggingface/accelerate.git
bitsandbytes
datasets==2.13.1
transformers @ git+https://github.com/huggingface/transformers.git
peft @ git+https://github.com/huggingface/peft.git
trl @ git+https://github.com/lvwerra/trl.git
scipy
```

Then install and import the installed libraries

In [1]:
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import networkx as nx

## Download LLaMA 2 model
As mentioned before, LLaMA 2 models come in different flavors which are 7B, 13B, and 70B. Your choice can be influenced by your computational resources. Indeed, larger models require more resources, memory, processing power, and training time.

To download the model you have been granted access to, **make sure you are logged in to the Hugging Face model hub**. As mentioned in the requirements step, you need to use the `huggingface-cli` login command.

The following function will help us to download the model and its tokenizer. It requires a bitsandbytes configuration that we will define later.

In [2]:
import os

base_dir = '/home/manish/thesis-implementations/quest_generation/llama2/'
model_name = 'meta-llama/llama-2-13b-hf'
model_dir = 'models'
model_path = os.path.join(base_dir, model_dir, model_name)

In [ ]:
print(model_path, os.path.exists(model_path))

/home/manish/thesis-implementations/quest_generation/llama2/models/meta-llama/llama-2-13b-hf True


In [3]:
def load_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token
    
    return tokenizer

In [5]:
def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{12288}MB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        use_cache=False,
        device_map="auto",
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    return model, load_tokenizer(model_name)

## Quest Dataset

Load the qust dataset for training and create prompts accordingly

In [25]:
import json
import numpy as np
import os
import random

dataset_path = os.path.join(base_dir, 'data')
train_file = 'train.jsonl'
val_file = 'val.jsonl'
data_files = {
	"train": train_file, 
	"val": val_file
}

## Create a bitsandbytes configuration and load the model and tokenizer
This will allow us to load our LLM in 4 bits. This way, we can divide the used memory by 4 and import the model on smaller devices. We choose to apply bfloat16 compute data type and nested quantization for memory-saving purposes.

In [5]:
def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

To leverage the LoRa method, we need to wrap the model as a PeftModel.

To do this, we need to implement a [LoRa configuration](https://huggingface.co/docs/peft/conceptual_guides/lora):

In [6]:
def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        lora_alpha=16,  # parameter for scaling
        lora_dropout=0.1,  # dropout probability for layers
        r=64,  # dimension of the updated matrices
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=modules,
    )

    return config

In [9]:
# Load model from HF with user's token and with bitsandbytes config
bnb_config = create_bnb_config()
model, tokenizer = load_model(model_path, bnb_config)
# tokenizer = load_tokenizer(model_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# TRAIN_TYPE = 'no_kg'
TRAIN_TYPE = 'text_kg'
# TRAIN_TYPE = 'tree_kg'
KG_DEPTH = 2

In [11]:
EOS_TOKEN = tokenizer.eos_token
PAD_TOKEN = tokenizer.pad_token
BOS_TOKEN = tokenizer.bos_token

In [12]:
map_game = {
    'TESO': 'TESOblivion_KG.gml',
    'TESS': 'TESSkyrim_KG.gml',
    'TL2': 'Torchlight2_KG.gml',
    'MC': 'Minecraft_KG.gml',
    'BG1': 'BaldursGate1_KG.gml',
    'BG2': 'BaldursGate2_KG.gml'
}

kg_map = {}

In [13]:
kg_base_dir = '/home/manish/thesis-implementations/data/VartinenFormatted/KGs'

for gid, gname in map_game.items():
    kg_path = os.path.join(kg_base_dir, map_game[gid])
    kg = nx.read_gml(kg_path)
    kg_map[gid] = kg

In [33]:
def create_training_prompt_formats(input):
    """
    Format various fields of the input quest data ('plots', 'kb', 'quest')
    Then concatenate them using two newline characters
    :param input: input dictionary
    """

    BACKGROUND = "### Background:"
    PLOTS_KEY = "### Plots:"
    INTRO_BLURB = "The quest related to the above information is as follows:"
    QUEST = "### Quest:"
    END_KEY = "### End"
    

    blurb = f"{INTRO_BLURB}"  # add intro blurb - model system instruction

    # add background - only if knowledge graph as text
    background = ''
    
    # add plots - key plot points
    plots_str = '\n'.join(input['plots'])
    plots = f"{PLOTS_KEY}\n{plots_str}"
    
    if TRAIN_TYPE == 'text_kg':
        completed_rels = []
        completed_nodes = []
        
        for kb in input['kbs']:
            entity = kb['name']
            e_desc = kb['description']
            e_type = kb['type']
            e_relations = kb['relations']
            
            background += f'{entity} is a {e_type}. '
            
            if entity != e_desc:
                background+= f'{entity} is {e_desc}. '
                
            for rel in e_relations:
                background += f'{entity} is {rel[0]} {rel[1]}.'
                completed_rels.append((entity, rel[1]))
            completed_nodes.append(entity)                     
            background += '\n'
        
        kg = kg_map[input['game']]
        all_nodes = kg.nodes(data=True)
        for node in all_nodes:
            entity = node[0]
            if entity.lower() in plots.lower():
                edges = list(nx.dfs_edges(kg, source=entity))
                for ent1, ent2 in edges:
                    if (ent1, ent2) in completed_rels or (ent2, ent1) in completed_rels:
                        continue
                    e1_type = all_nodes[ent1]['type']
                    e1_desc = all_nodes[ent1]['description']
                    e2_type = all_nodes[ent2]['type']
                    e2_desc = all_nodes[ent2]['description']
                    
                    if ent1 not in completed_nodes:
                        background += f'{ent1} is a {e1_type}. '
                        if e1_desc != ent1:
                            background += f'{ent1} is {e1_desc}. '
                        completed_nodes.append(ent1)
                        background += '\n'
                    
                    if ent2 not in completed_nodes:
                        background += f'{ent1} is a {e2_type}. '
                        if e2_desc != ent2:
                            background += f'{ent2} is {e2_desc}. '
                        completed_nodes.append(ent2)
                        background += '\n'
                    
                    rel = kg[ent1][ent2]['label']
                    if rel == 'connected to':
                        background += f'{ent1} is {rel} {ent2}. '
                    if rel == 'present in':
                        if e1_type == 'location':
                            background += f'{ent2} is {rel} {ent2}. '
                        else:
                            background += f'{ent1} is {rel} {ent2}. '
                    if rel == 'held by':
                        if e1_type == 'character':
                            background += f'{ent2} is {rel} {ent1}. '
                        else:
                            background += f'{ent1} is {rel} {ent2}. '
                    background += '\n'
                    completed_rels.append((ent1, ent2))
                        
        background = f"{BACKGROUND}\n{background}"
    
    # add concatenated quest text
    quest_str = ''
    for k,v in input['quest'].items():
        if k == 'description':
            continue
        if k == 'tasks':
            value = '\n ' + '\n '.join(np.char.capitalize(v[:-1]))
        else:
            value = v.capitalize()
        quest_str += f'{k.capitalize()}: {value}\n' 
    quest = f"{QUEST}\n{quest_str}"  # add quest output
    
    end = f"{END_KEY}"  # add end key
    
    if TRAIN_TYPE in ['no_kg', 'tree_kg']:
        parts = [part for part in [plots, blurb, quest, end] if part]
    else:
        parts = [part for part in [background, plots, blurb, quest, end] if part]

    formatted_prompt = "\n\n".join(parts)
    input['text'] = formatted_prompt

    return input

In [ ]:
# DO Direct relations, missed connections

In [31]:
def create_generation_prompt_formats(input):
    """
    Format various fields of the input quest data ('plots', 'kb', 'quest')
    Then concatenate them using two newline characters
    :param input: input dictionary
    """

    BACKGROUND = "### Background:"
    PLOTS_KEY = "### Plots:"
    INTRO_BLURB = "The quest related to the above information is as follows:"
    QUEST = "### Quest:"
    END_KEY = "### End"
    

    blurb = f"{INTRO_BLURB}"  # add intro blurb - model system instruction

    # add background - only if knowledge graph as text
    # create sentence tree from plots using kg - only if knowledge graph as tree
    background = ''
    plots_str = '\n'.join(input['plots'])
    plots = f"{PLOTS_KEY}\n{plots_str}"  # add plots - key plot points
    
    if TRAIN_TYPE == 'text_kg':
        completed_rels = []
        completed_nodes = []
        
        for kb in input['kbs']:
            entity = kb['name']
            e_desc = kb['description']
            e_type = kb['type']
            e_relations = kb['relations']
            
            background += f'{entity} is a {e_type}. '
            
            # add depth of information from KG
            # KG_DEPTH <= 0 will add all information
            # KG_DEPTH = 1 will only add description
            # KG_DEPTH > 1 will randomly add (KG_DEPTH - 1) number
            # of relations and the description
            if KG_DEPTH > 0:
                if entity != e_desc:
                    background+= f'{entity} is {e_desc}. '
                depth = KG_DEPTH - 1
                if depth > len(e_relations):
                    depth = len(e_relations)
                relations = random.sample(e_relations, depth)
            else:
                relations = e_relations
                
            for rel in relations:
                background += f'{entity} is {rel[0]} {rel[1]}. '
                completed_rels.append((entity, rel[1]))
            completed_nodes.append(entity)                     
            background += '\n'
        
        if KG_DEPTH > 0:
            kg = kg_map[input['game']]        
            all_nodes = kg.nodes(data=True)
            for node in all_nodes:
                entity = node[0]
                if entity.lower() in plots.lower():
                    edges = list(nx.dfs_edges(kg, source=entity, depth_limit=KG_DEPTH))
                    for ent1, ent2 in edges:
                        if (ent1, ent2) in completed_rels or (ent2, ent1) in completed_rels:
                            continue
                        e1_type = all_nodes[ent1]['type']
                        e1_desc = all_nodes[ent1]['description']
                        e2_type = all_nodes[ent2]['type']
                        e2_desc = all_nodes[ent2]['description']
                        
                        if ent1 not in completed_nodes:
                            background += f'{ent1} is a {e1_type}. '
                            if e1_desc != ent1:
                                background += f'{ent1} is {e1_desc}. '
                            completed_nodes.append(ent1)
                            background += '\n'
                        
                        if ent2 not in completed_nodes:
                            background += f'{ent2} is a {e2_type}. '
                            if e2_desc != ent2:
                                background += f'{ent2} is {e2_desc}. '
                            completed_nodes.append(ent2)
                            background += '\n'
                        
                        rel = kg[ent1][ent2]['label']
                        if rel == 'connected to':
                            background += f'{ent1} is {rel} {ent2}. '
                        if rel == 'present in':
                            if e1_type == 'location':
                                background += f'{ent2} is {rel} {ent2}. '
                            else:
                                background += f'{ent1} is {rel} {ent2}. '
                        if rel == 'held by':
                            if e1_type == 'character':
                                background += f'{ent2} is {rel} {ent1}. '
                            else:
                                background += f'{ent1} is {rel} {ent2}. '
                        background += '\n'
                        completed_rels.append((ent1, ent2))
                
        background = f"{BACKGROUND}\n{background}"
        
    # add concatenated quest text
    quest_str = ''
    for k,v in input['quest'].items():
        if k == 'description':
            continue
        if k == 'tasks':
            value = '\n ' + '\n '.join(np.char.capitalize(v[:-1]))
        else:
            value = v.capitalize()
        quest_str += f'{k.capitalize()}: {value}\n' 
    quest = f"{QUEST}\n{quest_str}"  # add quest output
    
    end = f"{END_KEY}"  # add end key
    
    if TRAIN_TYPE in ['no_kg', 'tree_kg']:
        parts_p = [part for part in [plots, blurb] if part]
    else:
        parts_p = [part for part in [background, plots, blurb] if part]

    parts_o = [part for part in [quest, end] if part]
    
    formatted_prompt = "\n\n".join(parts_p)
    formatted_output = "\n\n".join(parts_o)
    input['text'] = formatted_prompt
    input['output'] = formatted_output

    return input

In [35]:
out = create_generation_prompt_formats(val_dataset[7])
print(out['text'])

### Background:
Martin is a character. Martin is the player's friend. 
Jauffre is a character. Jauffre is the male Grandmaster of the Blades. 
Tiber Septim is a character. Tiber Septim is the legendary male Emperor who ascended to divinity. 
Imperial City is a location. Imperial City is a large city. 
Martin is present in Imperial City. 
Skingrad is a location. Skingrad is a city. 
Imperial City is connected to Skingrad. 
Rosethorn Hall is a location. Rosethorn Hall is the player's manor. 
Imperial City is connected to Rosethorn Hall. 
Arcane University is a location. Arcane University is the headquarters of the Mages Guild. 
Imperial City is connected to Arcane University. 
Lonely Suitor Lodge is a location. Lonely Suitor Lodge is an inn. 
Imperial City is connected to Lonely Suitor Lodge. 
Aleron Loche is a character. Aleron Loche is Ursanne's missing husband who developed a gambling addiction. 
Aleron Loche is present in Aleron Loche. 
Vinicia Melissaeia is a character. Vinicia Meli

In [18]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, dataset: str, include_kg: bool = True):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    :param include_kg (bool): Whether to include knowledge graph in the prompt
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_training_prompt_formats)#, batched=True)
    
    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["id", "game", "kbs", "plots", "quest"],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    return dataset

In [15]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_val_dataset(tokenizer: AutoTokenizer, max_length: int, dataset: str, include_kg: bool = True):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    :param include_kg (bool): Whether to include knowledge graph in the prompt
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_generation_prompt_formats)#, batched=True)
    
    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["id", "game", "kbs", "plots", "quest"],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    return dataset

Now, we will use the **model tokenizer to process these prompts into tokenized ones**.

The goal is to create input sequences of uniform length (which are suitable for fine-tuning the language model because it maximizes efficiency and minimize computational overhead), that must not exceed the model’s maximum token limit.

Previous function needs the target modules to update the necessary matrices. The following function will get them for our model:

In [17]:
# SOURCE https://github.com/artidoro/qlora/blob/main/qlora.py

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

Once everything is set up and the base model is prepared, we can use the `print_trainable_parameters()` helper function to see how many trainable parameters are in the model. We expect the LoRa model to have fewer trainable parameters compared to the original one, since we want to perform fine-tuning.

In [18]:
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

## Train
Now that everything is ready, we can pre-process our dataset and load our model using the set configurations:

In [26]:
dataset = load_dataset(dataset_path, data_files=data_files)
train_dataset = dataset['train']
val_dataset = dataset['val']

Found cached dataset json (/home/manish/.cache/huggingface/datasets/json/data-6359e290ba54d2fa/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
## Preprocess dataset

max_length = get_max_length(model)
# train_dataset = preprocess_dataset(tokenizer, max_length, train_dataset)
val_dataset = preprocess_val_dataset(tokenizer, max_length, val_dataset)

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Filter:   0%|          | 0/77 [00:00<?, ? examples/s]

In [ ]:
kg_map = None
import gc
gc.collect()

In [ ]:
def train(model, tokenizer, dataset, output_dir):
    # Apply preprocessing to the model to prepare it by
    # 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # 2 - Using the prepare_model_for_kbit_training method from PEFT
    model = prepare_model_for_kbit_training(model)

    # Get lora module names
    modules = find_all_linear_names(model)

    # Create PEFT config for these modules and wrap the model to PEFT
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)
    
    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)
    
    # Training parameters
    trainer = Trainer(
        model=model,
        train_dataset=dataset,
        args=TrainingArguments(
            output_dir=output_dir,
            overwrite_output_dir=True,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            optim="paged_adamw_32bit",
            logging_steps=20,
            learning_rate=2e-4,
            fp16=True,
            warmup_steps=10,
            max_steps=200,
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
    
    model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs
    
    ### SOURCE https://github.com/artidoro/qlora/blob/main/qlora.py
    # Verifying the datatypes before training
    
    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes: dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = 0
    for k, v in dtypes.items(): total+= v
    for k, v in dtypes.items():
        print(k, v, v/total)
    
    # Launch training
    print("Training...")
    
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    print(metrics)    
    
    # Saving model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok=True)
    trainer.model.save_pretrained(output_dir)
    
    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()
    
    
output_dir = os.path.join(base_dir, model_dir, 'results', model_name, f'{TRAIN_TYPE}_{KG_DEPTH}')
train(model, tokenizer, train_dataset, output_dir)

*If you prefer to have a number of epochs (entire training dataset will be passed through the model) instead of a number of training steps (forward and backward passes through the model with one batch of data), you can replace the `max_steps` argument by `num_train_epochs`.*

To later load and use the model for inference, we have used the `trainer.model.save_pretrained(output_dir)` function, which saves the fine-tuned model’s weights, configuration, and tokenizer files.

Unfortunately, it is possible that the latest weights are not the best. To solve this problem, you can implement a `EarlyStoppingCallback`, from transformers, during your fine-tuning. This will enable you to regularly test your model on the validation set, if you have one, and keep only the best weights.

## Merge weights
Once we have our fine-tuned weights, we can build our fine-tuned model and save it to a new directory, with its associated tokenizer. By performing these steps, we can have a memory-efficient fine-tuned model and tokenizer ready for inference!

In [9]:
output_dir = os.path.join(base_dir, model_dir, 'results', model_name, f'{TRAIN_TYPE}')
print(output_dir, os.path.exists(output_dir))

/home/manish/thesis-implementations/quest_generation/llama2/models/results/meta-llama/llama-2-13b-hf/text_kg True


In [52]:
del model
del tokenizer
import gc
gc.collect()

60618

In [10]:
# base_model = model

# from peft import PeftModel    
def load_pretrained_model(model_path, bnb_config, base_model_path):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{12288}MB'

    model = AutoPeftModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=bnb_config,
        device_map="auto",
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    
    return model, load_tokenizer(base_model_path)

bnb_config = create_bnb_config()
model, tokenizer = load_pretrained_model(output_dir, bnb_config, model_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
val_dataset[7]

{'id': 180,
 'game': 'TESO',
 'kbs': [{'name': 'Martin',
   'description': "the player's friend",
   'id': 'TESO336',
   'relations': [],
   'type': 'character'},
  {'name': 'Jauffre',
   'description': 'the male Grandmaster of the Blades',
   'id': 'TESO259',
   'relations': [],
   'type': 'character'},
  {'name': 'Tiber Septim',
   'description': 'the legendary male Emperor who ascended to divinity',
   'id': 'TESO555',
   'relations': [],
   'type': 'character'}],
 'plots': ["Martin needs the blood of a Divine for the ritual to open the portal to Camoran's Paradise in addition to the first item, the blood of a Daedra"],
 'quest': {'title': 'Blood of the Divines',
  'objective': 'recover the blood of a Divine for Martin',
  'tasks': ['talk to Jauffre about obtaining the blood of Tiber Septim',
   'recover the blood of a Divine for Martin'],
  'description': "I've figured out another item needed for the ritual to open the portal to Camoran's Paradise. The second item is the counterpar

In [24]:
print(val_dataset[7]['text'])

### Background:
Martin is a character. Martin is the player's friend. 
Jauffre is a character. Jauffre is the male Grandmaster of the Blades. 
Tiber Septim is a character. Tiber Septim is the legendary male Emperor who ascended to divinity. 
Martin is a location. Imperial City is a large city. 
Martin is present in Imperial City. 
Imperial City is a location. Skingrad is a city. 
Imperial City is connected to Skingrad. 
Imperial City is a location. Rosethorn Hall is the player's manor. 
Imperial City is connected to Rosethorn Hall. 
Imperial City is a location. Arcane University is the headquarters of the Mages Guild. 
Imperial City is connected to Arcane University. 
Imperial City is a location. Lonely Suitor Lodge is an inn. 
Imperial City is connected to Lonely Suitor Lodge. 
Imperial City is a character. Aleron Loche is Ursanne's missing husband who developed a gambling addiction. 
Aleron Loche is present in Aleron Loche. 
Imperial City is a character. Vinicia Melissaeia is Office 

## Inference

In [23]:
import gc
gc.collect()

5109

In [22]:
from tqdm.auto import tqdm
results = []

for item in tqdm(val_dataset):
	inp_txt_len = len(item['text'])
	print(inp_txt_len)
	# Specify input
	inp = torch.tensor([item['input_ids']])
	attn_mask = torch.tensor([item['attention_mask']])

	# Specify device
	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

	# Get answer
	# (Adjust max_new_tokens variable as you wish (maximum number of tokens the model can generate to answer the input))
	outputs = model.generate(
		input_ids=inp.to(device), 
		attention_mask=attn_mask, 
		max_new_tokens=250, 
		eos_token_id=tokenizer('### End')['input_ids'], 
		pad_token_id=tokenizer.eos_token_id,
  	)

	# Decode output & append to outptu list
	output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)[inp_txt_len + 2:]
	result = {'input': item['text'], 'output_gen': output_text, 'output_actual': item['output']}
	results.append(result)


  0%|          | 0/73 [00:00<?, ?it/s]

2045
3727
1417
2000
318
6122
594
7958


OutOfMemoryError: CUDA out of memory. Tried to allocate 472.00 MiB. GPU 0 has a total capacty of 15.68 GiB of which 193.44 MiB is free. Process 1846 has 239.88 MiB memory in use. Including non-PyTorch memory, this process has 14.76 GiB memory in use. Of the allocated memory 13.49 GiB is allocated by PyTorch, and 1005.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [48]:
print(results[3]['output_gen'])

### Quest:
Title: A gift for the queen
Objective: Bring a gift to the queen
Tasks: 
 Arrive at the blue palace.
 Give the queen the gift


### End


In [54]:
print(TRAIN_TYPE)

text_kg


In [50]:
out_file = os.path.join(base_dir, 'outputs', f'{TRAIN_TYPE}_{KG_DEPTH}_results.jsonl')
with open(out_file, 'w') as outfile:
	for result in results:
		json.dump(result, outfile)
		outfile.write('\n')

In [68]:
print(out_file)

/home/manish/thesis-implementations/quest_generation/llama2/outputs/text_kg_results.jsonl


In [ ]:
len(val_dataset[0]['attention_mask']) == len(val_dataset[0]['input_ids'])

In [57]:
def create_input_prompt(background_info, plots):
    text = ''
    if background_info:
        text += '### Background:\n\n'
        text += '\n'.join(background_info) + '\n\n'
    text += '### Plots:\n\n'
    text += "\n".join(plots) + '\n'
    text += '\n### Quest:\n'
    return text

In [76]:
bg_info = [
    # '',
	'Delilah is a character. Delilah is present in Brigmore Manor. Delilah is the leader of the Brigmore Witches.',
	'Brigmore Manor is a location. Brigmore Manor is connected to Empire of the Isles. Brigmore Manor is a lair for the city\'s outlaws.',
 	'Empire of the Isles is a location. Empire of the Isles is connected to Brigmore Manor. Empire of the Isles is a kingdom.',
]
plots = [
	'Delilah and her coven are planning something that threatens everyone across the Empire of the Isles.',
 	'Delilah must be stopped'
	# 'Infiltrate the ruins of Brigmore Manor and stop Delilah'
]

str_inp = create_input_prompt(bg_info, plots)
print(str_inp)

### Background:

Delilah is a character. Delilah is present in Brigmore Manor. Delilah is the leader of the Brigmore Witches.
Brigmore Manor is a location. Brigmore Manor is connected to Empire of the Isles. Brigmore Manor is a lair for the city's outlaws.
Empire of the Isles is a location. Empire of the Isles is connected to Brigmore Manor. Empire of the Isles is a kingdom.

### Plots:

Delilah and her coven are planning something that threatens everyone across the Empire of the Isles.
Delilah must be stopped

### Quest:



In [37]:
token_inp = tokenizer(out['text'])

In [38]:
inp = torch.tensor([token_inp['input_ids']])
attn_mask = torch.tensor([token_inp['attention_mask']])

# Specify device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Get answer
# (Adjust max_new_tokens variable as you wish (maximum number of tokens the model can generate to answer the input))
outputs = model.generate(
	input_ids=inp.to(device), 
	attention_mask=attn_mask, 
	max_new_tokens=250, 
	eos_token_id=tokenizer('### End')['input_ids'], 
	pad_token_id=tokenizer.eos_token_id,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1014.00 MiB. GPU 0 has a total capacty of 15.68 GiB of which 139.38 MiB is free. Process 1846 has 239.88 MiB memory in use. Including non-PyTorch memory, this process has 14.76 GiB memory in use. Of the allocated memory 12.05 GiB is allocated by PyTorch, and 2.43 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [39]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Background:
Gilgondorin is a character. Gilgondorin is a keeper of local legends. Gilgondorin is present in Niben Bay. 
Niben Bay is a location. Niben Bay is a sea bay. Niben Bay is connected to Bawnwatch Camp. 
Bawnwatch Camp is a location. Bawnwatch Camp is a camp on the shore of Niben Bay. 
Watchman is a character. Watchman is a harmless, sad ghost of a sailor. Watchman is present in Niben Bay. 


### Plots:
the player asked Gilgondorin about the Watchman
Gilgondorin pinpoints where the Watchman appears on the player's map

The quest related to the above information is as follows:

### Quest:
Title: A watchful eye
Objective: Talk to the watchman
Tasks: 
 Travel to bawnwatch camp.
 Find the watchman


### End


In [67]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Background:



### Plots:

Delilah and her coven are planning something that threatens everyone across the Empire of the Isles.
Infiltrate the ruins of Brigmore Manor and stop Delilah

### Quest:

Title: The brigmore conspiracy
Objective: Infiltrate the ruins of brigmore manor and stop delilah
Tasks: 
 Find your way to the ruins of brigmore manor


### End


In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
with open(os.path.join(base_dir, model_dir, 'results', model_name, f'{TRAIN_TYPE}_{KG_DEPTH}/results.txt'), 'w') as outfile:
	for result in results:
		outfile.write(f'{result}\n{"-"*40}\n')